In [17]:
import os

from firecrawl import FirecrawlApp
from groq import Groq

# initialize clients
groq_client = Groq(api_key=os.environ["GROQ_API_KEY"])
firecrawl_client = FirecrawlApp(api_key=os.environ["FIRECRAWL_API_KEY"])


In [2]:
chat_completion = groq_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is Firecrawl famous for?",
        }
    ],
    model="moonshotai/kimi-k2-instruct",
)

print(chat_completion.choices[0].message.content)

Firecrawl is best known for its ability to **turn entire websites into clean, LLM-ready markdown or structured data** in a single API call. It’s become popular among developers and AI teams who need to feed web content into large-language-model pipelines, RAG systems, or fine-tuning workflows without having to write custom scrapers or crawl logic.


In [15]:
# perform a basic search
search_result = firecrawl_client.search(
    "Find flight from Newyork to tokyo",
    limit=5,
    tbs="qdr:w",  # results from the past week
    timeout=30000,  # 30 seconds
)

for item in search_result.data:
    print("Title:", item["title"])
    print("URL:", item["url"])
    print("Description:", item["description"])
    print("-" * 40)


Title: American Airlines® - Find New York to Tokyo flights
URL: https://www.aa.com/en-us/flights-from-new-york-to-tokyo
Description: Find deals on New York to Tokyo flights ... *Fares displayed have been collected within the last 24hrs and may no longer be available at time of booking. Some ...
----------------------------------------
Title: Flights from New York to Tokyo (JFK - HND) - EVA Air
URL: https://flights.evaair.com/en/flights-from-new-york-to-tokyo
Description: Find New York to Tokyo Flights with EVA Air from USD 1,288. Round Trip ... Flight Type Round Trip|Economy. Dec 2025. No results available. Check ...
----------------------------------------
Title: Flights from New York to Tokyo | Singapore Airlines
URL: https://www.singaporeair.com/en-us/flights-from-new-york-to-tokyo
Description: Discover the best flight rates from New York to Tokyo. Travel in comfort with award-winning inflight services and state-of-the-art amenities.
----------------------------------------
Title: N

In [ ]:
# scrape a page in markdown & html
scrape_result = firecrawl_client.scrape_url(
    url="https://flights.evaair.com/en/flights-from-new-york-to-tokyo",
    formats=["markdown"],
    max_age=3600000,
)

# markdown content:
print(scrape_result.markdown[:500], "…")

[Skip to content](https://flights.evaair.com/en/flights-from-new-york-to-tokyo#main)- [Skip to footer](https://flights.evaair.com/en/flights-from-new-york-to-tokyo#footer)

# Find New York to Tokyo Flights with EVA Air from USD 1,288

Round Trip

_expand\_more_

1 Passenger, Economy

_expand\_more_

Departure City

_close_

Destination City

_close_

Departurefc-booking-departure-date-aria-label07/24/2025

_today_

Returnfc-booking-return-date-aria-label07/31/2025

_today_

Search

[![](https:// …


In [ ]:
def search_flights(query: str, limit: int = 5):
    results = firecrawl_client.search(
        query,
        limit=limit,
        tbs="qdr:w",  # results from the past week
        timeout=30000,
    )
    jobs = []
    for item in results.data:
        jobs.append(
            {
                "title": item["title"],
                "url": item["url"],
                "description": item["description"],
            }
        )
    return jobs



def summarize_flight(description: str) -> str:
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": f"""You are a helpful assistant. Summarize each flight search result into:
• Airline  
• Route (origin to destination)  
• Price (if available)  
• Key features or restrictions  
• Link to book flight
{description}""",
        },
    ]

    resp = groq_client.chat.completions.create(
        model="moonshotai/kimi-k2-instruct",
        messages=messages,
        max_tokens=512,
        temperature=0.7,
    )
    return resp.choices[0].message.content.strip()

In [31]:
def flight_search(query: str) -> str:
    flights = search_flights(query)
    summary = summarize_flight(str(flights))
    return summary


In [32]:
result = flight_search("Find flight from New York to Tokyo")
print(result)


• Airline: American Airlines  
  Route: New York (JFK/EWR/LGA) → Tokyo (NRT/HND)  
  Price: Not specified in search feed (“…may no longer be available…”)  
  Key features/restrictions: 24-hr price volatility warning; fare may change at booking  
  Link: https://www.aa.com/en-us/flights-from-new-york-to-tokyo  

• Airline: EVA Air  
  Route: New York (JFK) → Tokyo Haneda (HND)  
  Price: From USD 1,288 round-trip (Economy)  
  Key features/restrictions: Lowest price shown for Dec 2025 currently unavailable—check calendar  
  Link: https://flights.evaair.com/en/flights-from-new-york-to-tokyo  

• Airline: Singapore Airlines  
  Route: New York (JFK/EWR) → Tokyo (NRT/HND)  
  Price: Not displayed in search snippet  
  Key features/restrictions: Award-winning service, modern amenities, wide-body aircraft  
  Link: https://www.singaporeair.com/en-us/flights-from-new-york-to-tokyo  

• Airline: Air France  
  Route: New York (NYC-area airports) → Tokyo (TYO – both NRT/HND)  
  Price: “Best p